# Rayleigh–Taylor Instability

By: Jack B, Sahat J, and Riddhi C\
Last edited: _________

## Introduction
Rayleigh-Taylor instabilities (RTI) arise in fluid systems where a denser fluid overlays a less dense fluid. This configuration is inherently unstable, as the gravitational potential energy in the system can decrease if the denser fluid moves downward and the lighter fluid moves upward. If a perturbation is introduced at the interface between the fluids, the system responds by amplifying the disturbance, leading to the characteristic "fingers" or "spikes" of the denser fluid descending into the lighter fluid, accompanied by "bubbles" of the lighter fluid rising. \
\
RTI is a significant phenomenon because:
- In astrophysics, it helps explain mixing processes during supernova explosions and stellar evolution.
- In geophysics, it contributes to magma dynamics and mantle convection.
- In engineering, it provides a challenge in processes such as inertial confinement fusion, where controlling these instabilities is crucial for achieving stable energy confinement.


## Problem Description and Approach

Our goal is to model and simulate Rayleigh-Taylor instability numerically, allowing us to study the evolution of the interface and the fluid dynamics involved. To achieve this, we employ a combination of fluid dynamics equations and numerical methods.

## Mathematical Formulation

The evolution of the system is governed by several key equations:

### 1. Navier-Stokes Equations

These describe the motion of the fluid, incorporating viscosity, density, and external forces like gravity. The incompressible form of the Navier-Stokes equation is:
$$
\frac{\partial \vec{v}}{\partial t} + (\vec{v} \cdot \nabla)\vec{v} = -\frac{1}{\rho}\nabla P + \nu \nabla^2 \vec{v} + \vec{g}
$$
where $\vec{v} = \langle u, v \rangle$ is the velocity field, $\rho$ is the fluid density, $\nu$ is the kinematic viscosity, and $\vec{g}$ is the gravitational acceleration.

### 2. Continuity Equation

This ensures the incompressibility of the fluid:
$$
\nabla \cdot \vec{v} = 0
$$

### 3. Level Set Equation

To track the evolving interface between the two fluids, we use a scalar field $\phi(x, y, t)$ where the interface is located at $\phi = 0$. The level set method is used to evolve the interface:
$$
\frac{\partial \phi}{\partial t} + \vec{v} \cdot \nabla \phi = 0
$$

### 4. Density Field

The density of each fluid is determined by the level set function:
$$
\rho(x, y, t) = \rho_{\text{light}} + (\rho_{\text{heavy}} - \rho_{\text{light}}) H(\phi(x, y, t))
$$
where $H(\phi)$ is the Heaviside step function, which is -1 when $\phi < 0$, 0 when $\phi = 0$, and 1 when $\phi > 0$.

### 5. Vorticity Equation

The vorticity $\omega$ of the fluid flow, which represents the rotational component of the velocity field, is calculated by:
$$
\frac{\partial \omega}{\partial t} + \vec{v} \cdot \nabla \omega = \nu \nabla^2 \omega + \frac{g}{\rho} \frac{\partial \rho}{\partial y}
$$
This equation governs the evolution of vorticity in the fluid.

### 6. Velocity Recovery

The velocity field $\vec{v}$ is recovered from the vorticity $\omega$ by solving the Poisson equation:
$$
\nabla^2 \psi = -\omega
$$
Then, the velocity components are derived from the stream function $\psi$:
$$
u = \frac{\partial \psi}{\partial y}, \quad v = -\frac{\partial \psi}{\partial x}
$$

## Numerical Methods

### 1. Grid Initialization

We begin by creating a uniform 2D grid $(x, y)$, representing the simulation space. The density field $\rho(x, y)$ is initialized with a smooth transition between the two fluids using a hyperbolic tangent function:
$$
\rho(x, y, 0) = \rho_{\text{light}} + (\rho_{\text{heavy}} - \rho_{\text{light}}) \frac{1}{2} \left( 1 + \tanh\left( y - \frac{y_{\text{interface}}}{\delta} \right) \right)
$$

### 2. Level Set Function Initialization

The level set field $\phi(x, y)$ is initialized to represent the interface:
$$
\phi(x, y) = y - y_{\text{interface}}
$$
where $y_{\text{interface}}$ is the initial position of the fluid interface.

### 3. Advection of the Level Set Function

To update the position of the interface over time, the level set function $\phi$ is evolved using the velocity field:
$$
\phi^{n+1}_{i,j} = \phi^{n}_{i,j} - \Delta t \left( u_{i,j} \frac{\partial \phi}{\partial x} + v_{i,j} \frac{\partial \phi}{\partial y} \right)
$$

### 4. Reinitialization of the Level Set Function

After each time step, we reinitialize the level set function to ensure it remains a signed distance function:
$$
\frac{\partial \phi}{\partial \tau} = \text{sign}(\phi_0)(1 - |\nabla \phi|)
$$
This step maintains the stability and accuracy of the interface representation.

### 5. Density Update

The density field $\rho$ is updated by applying the Heaviside function $H(\phi)$, which maps regions above the interface to $\rho_{\text{light}}$ and regions below the interface to $\rho_{\text{heavy}}$.

### 6. Vorticity Update

The vorticity equation is solved to update the vorticity field $\omega$, which describes the rotational motion of the fluid.

### 7. Poisson Equation for Velocity Recovery

The stream function $\psi$ is computed by solving the Poisson equation:
$$
\nabla^2 \psi = -\omega
$$
The velocity components $u$ and $v$ are then derived from $\psi\$.

### 8. Visualization

To visualize the simulation, we plot the zero-contour of the level set function $\phi$, which marks the interface between the two fluids. 

In [19]:
# Code Goes Here
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
plt.switch_backend("Qt5Agg")

# --- Advection Function ---
def advect_phi(phi, u, v, dx, dy, dt):
    cfl_dt = min(dx / (np.abs(u).max() + 1e-6), dy / (np.abs(v).max() + 1e-6))
    dt = min(dt, cfl_dt)
    dphi_dx = np.gradient(phi, dx, axis=1)
    dphi_dy = np.gradient(phi, dy, axis=0)
    phi_new = phi - dt * (u * dphi_dx + v * dphi_dy)
    return phi_new

# --- Reinitialization Function ---
def reinitialize_phi(phi, dx, dy, tau=0.1, iterations=50):
    epsilon = 1e-6
    for _ in range(iterations):
        dphi_dx = (np.roll(phi, -1, axis=1) - np.roll(phi, 1, axis=1)) / (2 * dx)
        dphi_dy = (np.roll(phi, -1, axis=0) - np.roll(phi, 1, axis=0)) / (2 * dy)
        grad_phi = np.sqrt(dphi_dx**2 + dphi_dy**2 + epsilon**2)
        sign_phi = phi / np.sqrt(phi**2 + epsilon**2)
        phi += tau * sign_phi * (1 - grad_phi)
    return phi

# --- Density Update Function ---
def heaviside(phi, epsilon=1e-6):
    return 0.5 * (1 + phi / np.sqrt(phi**2 + epsilon**2))

def update_density(phi, rho_light, rho_heavy):
    H = heaviside(phi)
    return rho_light + (rho_heavy - rho_light) * H

# --- Vorticity Update ---
def update_vorticity(omega, u, v, rho, dx, dy, nu, g, dt):
    d_omega_dx = (np.roll(omega, -1, axis=1) - np.roll(omega, 1, axis=1)) / (2 * dx)
    d_omega_dy = (np.roll(omega, -1, axis=0) - np.roll(omega, 1, axis=0)) / (2 * dy)
    d_rho_dx = (np.roll(rho, -1, axis=1) - np.roll(rho, 1, axis=1)) / (2 * dx)
    laplacian_omega = (
        (np.roll(omega, -1, axis=1) - 2 * omega + np.roll(omega, 1, axis=1)) / dx**2 +
        (np.roll(omega, -1, axis=0) - 2 * omega + np.roll(omega, 1, axis=0)) / dy**2
    )
    advection = u * d_omega_dx + v * d_omega_dy
    diffusion = nu * laplacian_omega
    baroclinic = g * d_rho_dx / np.maximum(rho, 1e-6)
    omega_new = omega + dt * (-advection + diffusion + baroclinic)
    return omega_new

# --- Poisson Solver ---
def solve_poisson_sor(omega, dx, dy, tol=1e-3, max_iter=5000, omega_relax=1.5):
    Ny, Nx = omega.shape
    psi = np.zeros_like(omega)
    for iteration in range(max_iter):
        psi_old = psi.copy()
        for i in range(1, Ny-1):
            for j in range(1, Nx-1):
                psi[i, j] = (1 - omega_relax) * psi[i, j] + omega_relax / (2 / dx**2 + 2 / dy**2) * (
                    (psi[i+1, j] + psi[i-1, j]) / dx**2 +
                    (psi[i, j+1] + psi[i, j-1]) / dy**2 - omega[i, j]
                )
        psi[0, :] = psi[-1, :] = psi[:, 0] = psi[:, -1] = 0
        if np.linalg.norm(psi - psi_old, ord=np.inf) < tol:
            break
    return psi

# --- Velocity Update ---
def update_velocities(psi, dx, dy):
    u = np.gradient(psi, axis=1) / dy
    v = -np.gradient(psi, axis=0) / dx
    return u, v

# --- Simulation Parameters ---
Nx, Ny = 40, 256
dx, dy = 1.0, 1.0
dt = 0.01
time_steps = 100
nu = 20.0
g = 9.81
rho_light = 1.0
rho_heavy = 2.0

# --- Initial Conditions ---
x = np.arange(Nx)
y_perturb = Ny // 2 + 10 * np.cos(2 * np.pi * x / 20)
phi = np.zeros((Ny, Nx))
for i in range(Nx):
    phi[:int(y_perturb[i]), i] = 1
u = np.zeros((Ny, Nx))
v = np.zeros((Ny, Nx))
omega = np.zeros((Ny, Nx))
rho = update_density(phi, rho_light, rho_heavy)

In [21]:
# Figures and Animations
# --- Visualization Setup ---
fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(phi, extent=[0, Nx * dx, 0, Ny * dy], origin="lower", cmap="coolwarm", vmin=0, vmax=1)
ax.set_title("Rayleigh-Taylor Instability")
ax.set_xlabel("x")
ax.set_ylabel("y")

# --- Animation Function ---
def animate(frame):
    global phi, u, v, omega, rho
    phi = advect_phi(phi, u, v, dx, dy, dt)
    phi = reinitialize_phi(phi, dx, dy)
    rho = update_density(phi, rho_light, rho_heavy)
    omega = update_vorticity(omega, u, v, rho, dx, dy, nu, g, dt)
    psi = solve_poisson_sor(omega, dx, dy)
    u, v = update_velocities(psi, dx, dy)
    im.set_array(phi)
    return [im]

anim = FuncAnimation(fig, animate, frames=time_steps, interval=50, blit=True)
plt.show()

`Discussing the results. Does it make sense? What do you learn?`

References

Contribution of each team member